In [1]:
import pandas as pd


In [2]:
train_events = pd.read_csv("../data/raw/train_events.csv")
train_events.groupby('series_id').head(1)["event"].unique()
train_events.groupby('series_id').tail(1)["event"].unique()


array(['wakeup'], dtype=object)

In [3]:
series_has_NaN = train_events.groupby('series_id')['step'].apply(lambda x: x.isnull().any())
series_has_NaN.value_counts()

step
True     240
False     37
Name: count, dtype: int64

In [4]:
no_NaN_series = series_has_NaN[~series_has_NaN].index.tolist()
no_NaN_series

['08db4255286f',
 '0a96f4993bd7',
 '0cfc06c129cc',
 '1087d7b0ff2e',
 '10f8bc1f7b07',
 '18b61dd5aae8',
 '29c75c018220',
 '31011ade7c0a',
 '3452b878e596',
 '349c5562ee2c',
 '3664fe9233f9',
 '483d6545417f',
 '55a47ff9dc8a',
 '5acc9d63b5fd',
 '5f94bb3e1bed',
 '655f19eabf1e',
 '67f5fc60e494',
 '72bbd1ac3edf',
 '76237b9406d5',
 '7822ee8fe3ec',
 '89bd631d1769',
 '8e32047cbc1f',
 '939932f1822d',
 '9ee455e4770d',
 'a596ad0b82aa',
 'a9a2f7fac455',
 'a9e5f5314bcb',
 'af91d9a50547',
 'b364205aba43',
 'c535634d7dcd',
 'c6788e579967',
 'c68260cc9e8f',
 'ca730dbf521d',
 'd150801f3145',
 'd25e479ecbb7',
 'd515236bdeec',
 'd5e47b94477e']

In [5]:
# also drop these two "truncated" events series seen in EDA:
no_NaN_series.remove('31011ade7c0a') # incomplete events data
no_NaN_series.remove('a596ad0b82aa') # incomplete events data

In [6]:
def get_train_series(series):
    train_series = pd.read_parquet("../data/raw/train_series.parquet", filters=[('series_id','=',series)])
    train_events = pd.read_csv("../data/raw/train_events.csv").query('series_id == @series')
    
    train_events = train_events.dropna()
    train_events["step"]  = train_events["step"].astype("int")
    train_events["events"] = train_events["event"].replace({"onset":1,"wakeup":0})

    train = pd.merge(train_series, train_events[['step','events']], on='step', how='left')
    train["events"] = train["events"].bfill(axis ='rows')
    # final section:
    # train_events.groupby('series_id').tail(1)["event"].unique()
    # Result: the last event is always a "wakeup"
    train['events'] = train['events'].fillna(1) # events
    train["events"] = train["events"].astype("int")
    return(train)

In [7]:
smaller_train_data = []

for series_id in no_NaN_series:
    train = get_train_series(series_id)
    smaller_train_data.append(train)

/tmp/ipykernel_133196/3670474102.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_events["events"] = train_events["event"].replace({"onset":1,"wakeup":0})
/tmp/ipykernel_133196/3670474102.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_events["events"] = train_events["event"].replace({"onset":1,"wakeup":0})
/tmp/ipykernel_133196/3670474102.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects

In [8]:
Zzzs_train = pd.concat(smaller_train_data).reset_index(drop=True)
Zzzs_train["series_id"].nunique()


35

In [9]:
Zzzs_train


,series_id,step,timestamp,anglez,enmo,events
0,08db4255286f,0,2018-11-05T10:00:00-0400,-30.845301,0.0447,1
1,08db4255286f,1,2018-11-05T10:00:05-0400,-34.181801,0.0443,1
2,08db4255286f,2,2018-11-05T10:00:10-0400,-33.877102,0.0483,1
3,08db4255286f,3,2018-11-05T10:00:15-0400,-34.282101,0.0680,1
4,08db4255286f,4,2018-11-05T10:00:20-0400,-34.385799,0.0768,1
...,...,...,...,...,...,...
13165555,d5e47b94477e,395815,2017-12-02T15:44:35-0500,-10.064100,0.1552,1
13165556,d5e47b94477e,395816,2017-12-02T15:44:40-0500,-10.006700,0.1139,1
13165557,d5e47b94477e,395817,2017-12-02T15:44:45-0500,-3.328600,0.1245,1
13165558,d5e47b94477e,395818,2017-12-02T15:44:50-0500,-5.903100,0.1363,1


In [10]:
Zzzs_train.to_parquet('../data/processed/smaller_data.parquet')